In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [ ]:
# Select the last unique 27 active batter ideas 

df = pd.read_csv('kbo-hitter-strikeout-rates.csv')
df['strikeout_rate'] = df['strikeouts']/df['plate_appearances']
df['strikeout_rate'] = df['strikeout_rate'].fillna(0)
df1 = df.sort_values('game_id')
df1 = df[df['season']==2021]

df1

In [ ]:
batter_id_values = df1[['batter_id']].values
unique_batter_id_values =  np.unique(batter_id_values)
batter_id_2022 = unique_batter_id_values[-27:]
List_of_Batters = batter_id_2022.tolist()
print(List_of_Batters)

In [ ]:
full_data = df[df['batter_id'].isin(List_of_Batters)]

In [ ]:
#Split the data into Each Batter

data_player1 = full_data[full_data['batter_id']==batter_id_2022[0]]
data_player2 = full_data[full_data['batter_id']==batter_id_2022[1]]
data_player3 = full_data[full_data['batter_id']==batter_id_2022[2]]
data_player4 = full_data[full_data['batter_id']==batter_id_2022[3]]
data_player5 = full_data[full_data['batter_id']==batter_id_2022[4]]
data_player6 = full_data[full_data['batter_id']==batter_id_2022[5]]
data_player7 = full_data[full_data['batter_id']==batter_id_2022[6]]
data_player8 = full_data[full_data['batter_id']==batter_id_2022[7]]
data_player9 = full_data[full_data['batter_id']==batter_id_2022[8]]
data_player10 = full_data[full_data['batter_id']==batter_id_2022[9]]
data_player11 = full_data[full_data['batter_id']==batter_id_2022[10]]
data_player12 = full_data[full_data['batter_id']==batter_id_2022[11]]
data_player13 = full_data[full_data['batter_id']==batter_id_2022[12]]
data_player14 = full_data[full_data['batter_id']==batter_id_2022[13]]
data_player15 = full_data[full_data['batter_id']==batter_id_2022[14]]
data_player16 = full_data[full_data['batter_id']==batter_id_2022[15]]
data_player17 = full_data[full_data['batter_id']==batter_id_2022[16]]
data_player18 = full_data[full_data['batter_id']==batter_id_2022[17]]
data_player19 = full_data[full_data['batter_id']==batter_id_2022[18]]
data_player20 = full_data[full_data['batter_id']==batter_id_2022[19]]
data_player21 = full_data[full_data['batter_id']==batter_id_2022[20]]
data_player22 = full_data[full_data['batter_id']==batter_id_2022[21]]
data_player23 = full_data[full_data['batter_id']==batter_id_2022[22]]
data_player24 = full_data[full_data['batter_id']==batter_id_2022[23]]
data_player25 = full_data[full_data['batter_id']==batter_id_2022[24]]
data_player26 = full_data[full_data['batter_id']==batter_id_2022[25]]
data_player27 = full_data[full_data['batter_id']==batter_id_2022[26]]

In [ ]:
def Model(data,size,plot_name):
    #Remove the data the we wish to model and reshape it to be used in the model

    data = data.reset_index()['strikeout_rate']
    data = np.array(data).reshape(-1,1)
    
    #Splitting dataset into train and test split (65:35 split)

    training_size = int(len(data)*0.65)
    test_size = len(data)-training_size
    train_data, test_data = data[0:training_size,:], data[training_size:len(data),:1]
    
    #Function to convert an array of values into a dataset matrix

    def create_dataset(dataset, time_step=1):
        dataX, dataY = [],[]
        for i in range(len(dataset)-time_step-1):
            a = dataset[i:(i+time_step),0]
            dataX.append(a)
            dataY.append(dataset[i+time_step,0])
        return np.array(dataX), np.array(dataY)
    
    #Use the made function to Create the Datatset

    time_step = size
    X_train, y_train = create_dataset(train_data, time_step)
    X_test, y_test = create_dataset(test_data, time_step)
    
    #Reshape into to be (Samples, Time steps, features) for LSTM

    X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
    
    #Create a LSTM Model
    #Using three LSTM layers and one Dense layer

    model = Sequential()
    model.add(LSTM(50, return_sequences=True,input_shape = (size,1)))
    model.add(LSTM(50, return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(1))
    model.compile(loss = 'mean_squared_error', optimizer = 'adam')
    
    #Fit the model using the Training Data and use the Test Data to validate the model

    model.fit(X_train, y_train, validation_data = (X_test,y_test),epochs = 5, batch_size =256, verbose=1)

    #Predict and Test the model for performance metrics

    train_predict = model.predict(X_train)
    test_predict = model.predict(X_test)
    print('RMSE Training Data: ')
    print(math.sqrt(mean_squared_error(y_train, train_predict)))
    print('RMSE Test Data: ')
    print(math.sqrt(mean_squared_error(y_test, test_predict)))
   
    x_input = test_data[(len(test_data)-size):].reshape(1,-1)
    temp_input = list(x_input)
    temp_input = temp_input[0].tolist()
    x_input.shape
    temp_input
    
    lst_output = []
    n_steps = size
    i=0
    while (i<6):
        if(len(temp_input)>size):

            #print(temp_input)
            x_input = np.array(temp_input[1:])
            x_input = x_input.reshape(1,-1)
            x_input = x_input.reshape((1,n_steps,1))

            #print(x_input)

            yhat = model.predict(x_input, verbose = 1)
            temp_input.extend(yhat[0].tolist())
            temp_input = temp_input[1:]

            #print(temp_input)

            lst_output.extend(yhat.tolist())
            i = i+1
        else:
            x_input = x_input.reshape((1, n_steps,1))
            yhat = model.predict(x_input,verbose =1)
            #print(yhat[0])
            temp_input.extend(yhat[0].tolist())
            #print(len(temp_input))
            lst_output.extend(yhat.tolist())
            i = i+1

    print(lst_output)
    
    game_new = np.arange(1,(size + 1))
    game_pred = np.arange((size + 1),(size + 7))

    df_pred = data.tolist()
    df_pred.extend(lst_output)

    plt.figure(figsize=(18, 6))
    plt.plot(game_new,data[(len(data)-size):])

    plt.plot(game_pred,lst_output)
    plt.xlabel('Games')
    plt.ylabel('Strikout Rate (K%)')
    
    plt.savefig(plot_name)
    
    return plt




In [ ]:

model_player1 = Model(data_player1,100,"Strikeout_Rate_Pred 1.jpeg")
model_player2 = Model(data_player2,100,"Strikeout_Rate_Pred 2.jpeg")
model_player3 = Model(data_player3,100,"Strikeout_Rate_Pred 3.jpeg")
model_player4 = Model(data_player4,100,"Strikeout_Rate_Pred 4.jpeg")
model_player5 = Model(data_player5,100,"Strikeout_Rate_Pred 5.jpeg")
model_player6 = Model(data_player6,100,"Strikeout_Rate_Pred 6.jpeg")
model_player7 = Model(data_player7,100,"Strikeout_Rate_Pred 7.jpeg")
model_player8 = Model(data_player8,100,"Strikeout_Rate_Pred 8.jpeg")
model_player9 = Model(data_player9,100,"Strikeout_Rate_Pred 9.jpeg")
model_player10 = Model(data_player10,100,"Strikeout_Rate_Pred 10.jpeg")
model_player11 = Model(data_player11,100,"Strikeout_Rate_Pred 11.jpeg")
model_player12 = Model(data_player12,100,"Strikeout_Rate_Pred 12.jpeg")
model_player13 = Model(data_player13,100,"Strikeout_Rate_Pred 13.jpeg")
model_player14 = Model(data_player14,100,"Strikeout_Rate_Pred 14.jpeg")
model_player15 = Model(data_player15,100,"Strikeout_Rate_Pred 15.jpeg")
model_player16 = Model(data_player16,100,"Strikeout_Rate_Pred 16.jpeg")
model_player17 = Model(data_player17,100,"Strikeout_Rate_Pred 17.jpeg")
model_player18 = Model(data_player18,100,"Strikeout_Rate_Pred 18.jpeg")
model_player19 = Model(data_player19,100,"Strikeout_Rate_Pred 19.jpeg")
model_player20 = Model(data_player20,100,"Strikeout_Rate_Pred 20.jpeg")
model_player21 = Model(data_player21,100,"Strikeout_Rate_Pred 21.jpeg")
model_player22 = Model(data_player22,100,"Strikeout_Rate_Pred 22.jpeg")
model_player23 = Model(data_player23,50,"Strikeout_Rate_Pred 23.jpeg")
model_player24 = Model(data_player24,100,"Strikeout_Rate_Pred 24.jpeg")
model_player25 = Model(data_player25,100,"Strikeout_Rate_Pred 25.jpeg")
model_player26 = Model(data_player26,100,"Strikeout_Rate_Pred 26.jpeg")
model_player27 = Model(data_player27,100,"Strikeout_Rate_Pred 27.jpeg")
